<a href="https://colab.research.google.com/github/cavendano01/Statistical-Learning-I/blob/main/ensemble_learning/proyecto_titanic_parte_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Parte 1: Entrenamiento y validación y selección
(Nivel de exactitud mínimo deseado: 80%)

**Nota:** 

En los casos en donde se deba calcular métricas de evaluación,con objetivos de simplificar el problema se puede usar cualquier herramienta , por ejemplo sklearn en el subpaquete metrics:

https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html

https://scikit-learn.org/stable/modules/model_evaluation.html


## Preparación de Notebook & Datos

### Agregando Librerías requeridas

In [41]:
import pandas as pd
import numpy as np # Data Manipulation & Naive Bayes
import matplotlib.pyplot as plt #Data Exploration
import tensorflow as tf #Logistic Regression
from sklearn.tree import DecisionTreeClassifier, export_graphviz #SkLearn decission Tree
# https://github.com/andressa/sklearn-decisiontree/blob/master/Ensemble%20models%20with%20Decision%20Trees.ipynb
from sklearn import svm #Import svm model
# https://www.datacamp.com/community/tutorials/svm-classification-scikit-learn-python

from sklearn.model_selection import train_test_split # Spliting Data 
print ("Packages Loaded")

from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()

from sklearn.linear_model import LogisticRegression

from sklearn.svm import SVC

from sklearn.metrics import accuracy_score

from sklearn import model_selection

from sklearn.ensemble import VotingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
import joblib

Packages Loaded


### Ajustando TF para compatibilidad con TF1

In [2]:
if tf.__version__.startswith("2."):
  import tensorflow.compat.v1 as tf
  tf.compat.v1.disable_v2_behavior()
  tf.compat.v1.disable_eager_execution()

Instructions for updating:
non-resource variables are not supported in the long term


### Cargando dataset como array de numpy

In [3]:
#Data Loading
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [4]:
titanic =pd.read_csv("/content/drive/MyDrive/Statistical Learning O/proyecto_final/data_titanic_proyecto.csv")

In [5]:
titanic.head(10)

,PassengerId,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,passenger_class,passenger_sex,passenger_survived
0,1,"Braund, Mr. Owen Harris",22.0,1,0,A/5 21171,7.2500,NaN,S,Lower,M,N
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,PC 17599,71.2833,C85,C,Upper,F,Y
2,3,"Heikkinen, Miss. Laina",26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Lower,F,Y
3,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,1,0,113803,53.1000,C123,S,Upper,F,Y
4,5,"Allen, Mr. William Henry",35.0,0,0,373450,8.0500,NaN,S,Lower,M,N
5,6,"Moran, Mr. James",NaN,0,0,330877,8.4583,NaN,Q,Lower,M,N
6,7,"McCarthy, Mr. Timothy J",54.0,0,0,17463,51.8625,E46,S,Upper,M,N
7,8,"Palsson, Master. Gosta Leonard",2.0,3,1,349909,21.0750,NaN,S,Lower,M,N
8,9,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",27.0,0,2,347742,11.1333,NaN,S,Lower,F,Y
9,10,"Nasser, Mrs. Nicholas (Adele Achem)",14.0,1,0,237736,30.0708,NaN,C,Middle,F,Y


In [6]:
#Count the number of rows and columns in the data set 
titanic.shape

(891, 12)

In [7]:
titanic.describe()

,PassengerId,Age,SibSp,Parch,Fare
count,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,29.699118,0.523008,0.381594,32.204208
std,257.353842,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,38.000000,1.000000,0.000000,31.000000
max,891.000000,80.000000,8.000000,6.000000,512.329200


## Feature Engineering 

### Lidiando con NAs

In [8]:
titanic.isna().sum()

PassengerId             0
Name                    0
Age                   177
SibSp                   0
Parch                   0
Ticket                  0
Fare                    0
Cabin                 687
Embarked                2
passenger_class         0
passenger_sex           0
passenger_survived      0
dtype: int64

#### Columna Cabin
Ya que existen 687 datos, utilizar imputación de promedio puede sesgar el dataset, especialemtne ya que tenemos una idea de la categoría del boleto. Procedemos a eliminar columna de análisis.

In [9]:
titanic = titanic.drop(['Cabin'], axis=1)
titanic.isna().sum()

PassengerId             0
Name                    0
Age                   177
SibSp                   0
Parch                   0
Ticket                  0
Fare                    0
Embarked                2
passenger_class         0
passenger_sex           0
passenger_survived      0
dtype: int64

#### Columna Age 
Tenemos 177 resultados de Age que tenemos que cambiar, para no perder las demás características en el modelo imputaremos la edad promedio

In [10]:
titanic = titanic.fillna(titanic.mean())
titanic.isna().sum()

PassengerId           0
Name                  0
Age                   0
SibSp                 0
Parch                 0
Ticket                0
Fare                  0
Embarked              2
passenger_class       0
passenger_sex         0
passenger_survived    0
dtype: int64

### Lidiando con columnas No Numéericas

In [11]:
titanic.dtypes

PassengerId             int64
Name                   object
Age                   float64
SibSp                   int64
Parch                   int64
Ticket                 object
Fare                  float64
Embarked               object
passenger_class        object
passenger_sex          object
passenger_survived     object
dtype: object

ya que tenemos atributos como Sexo y edad, el nombre de una persona es redundante , pudiera ser

In [12]:
#titanic = titanic.drop(['Cabin'], axis=1)

### Variable Encoding: 
* Embarked, 
* passenger_sex 
* passenger_class
* Passenger_survived

In [13]:
#Encode embarked
titanic['Embarked'] = titanic['Embarked'].astype('str') 
titanic.iloc[:,7]= labelencoder.fit_transform(titanic.iloc[:,7].values)
#print(labelencoder.fit_transform(titanic.iloc[:,7].values))

In [14]:
#Encoding Sex como 0 y 1
titanic.iloc[:,9]= labelencoder.fit_transform(titanic.iloc[:,9].values)
#print(labelencoder.fit_transform(titanic.iloc[:,2].values))



In [15]:
#Encode passenger_class
titanic['passenger_class'] = titanic['passenger_class'].astype('str') 
titanic.iloc[:,8]= labelencoder.fit_transform(titanic.iloc[:,8].values)
#print(labelencoder.fit_transform(titanic.iloc[:,7].values))

In [16]:
#Encode Passenger_survived
titanic['passenger_survived'] = titanic['passenger_survived'].astype('str') 
titanic.iloc[:,10]= labelencoder.fit_transform(titanic.iloc[:,10].values)
#print(labelencoder.fit_transform(titanic.iloc[:,7].values))

In [17]:
#Print the NEW unique values in the columns
print(titanic['passenger_sex'].unique())
print(titanic['Embarked'].unique())
print(titanic['passenger_class'].unique())
print(titanic['passenger_survived'].unique())

[1 0]
[2 0 1 3]
[0 2 1]
[0 1]


In [18]:
titanic.dtypes

PassengerId             int64
Name                   object
Age                   float64
SibSp                   int64
Parch                   int64
Ticket                 object
Fare                  float64
Embarked                int64
passenger_class         int64
passenger_sex           int64
passenger_survived      int64
dtype: object

### Additional checks

In [19]:
titanic.isna().sum()

PassengerId           0
Name                  0
Age                   0
SibSp                 0
Parch                 0
Ticket                0
Fare                  0
Embarked              0
passenger_class       0
passenger_sex         0
passenger_survived    0
dtype: int64

In [20]:
titanic.dtypes

PassengerId             int64
Name                   object
Age                   float64
SibSp                   int64
Parch                   int64
Ticket                 object
Fare                  float64
Embarked                int64
passenger_class         int64
passenger_sex           int64
passenger_survived      int64
dtype: object

#### Drop de Tickets ya que no posee valore repetitivos a encoding

In [21]:
titanic = titanic.drop(['Ticket'], axis=1)
titanic.dtypes

PassengerId             int64
Name                   object
Age                   float64
SibSp                   int64
Parch                   int64
Fare                  float64
Embarked                int64
passenger_class         int64
passenger_sex           int64
passenger_survived      int64
dtype: object

## Train-val-test split 
* Conjunto de entrenamiento 
* Conjunto de validación
* Conjunto de pruebas. en csv


### Definiendo Variable dependiente Y y variables independientes

In [22]:
titanic.shape
#titanic.head(10)

(891, 10)

In [23]:
X = titanic.iloc[:, 2:8].values #Excluyendo Nombre y ID de variables predictivas
y = titanic.iloc[:, 9].values 

#print(X)
#print(y)

### Spliting Data en Train, Val, Test

In [24]:
#Split 1 para obtener el test Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
#Split 12 para obtener el validation data
X_train, X_val, y_train, y_val   = train_test_split(X_train, y_train, test_size=0.25, random_state=1) # 0.25 x 0.8 = 0.2

In [25]:
print("Training Data:", X_train.shape)
print("Validation Data:", X_val.shape)
print("Testing Data:", X_test.shape)


Training Data: (534, 6)
Validation Data: (178, 6)
Testing Data: (179, 6)


### Printing Test to CSV to use in "production

In [94]:
pd.DataFrame(X_test).to_csv("/content/drive/MyDrive/Statistical Learning O/proyecto_final/X_test.csv")
pd.DataFrame(y_test).to_csv("/content/drive/MyDrive/Statistical Learning O/proyecto_final/y_test.csv")

## Ensemble learning:
Se aplicará ensemble learning para crear una "votación mayoritaria" con distintos tipos de modelos, con el objetivo de simplificar el problema 2 de estos modelos se harán usando scikit-learn y su función   .fit(x,y) 

**Recordar  hacer muestreo bootstrap**

El ensamble estará compuesto de : 

*   Árbol de decisión con sklearn
*   SVM con sklearn
*   Naive bayes con numpy y/o pandas
*   Reg. logística binaria(sigmoid): en Tensorflow con regularización (probar L1, L2 y distintos valores del factor de regularización y elegir el mejor) y mini-batch gradient descent usando tamaño de mini-batch como hyper-parametro.

Se recomienda crear una función de Python para el entrenamiento de  cada tipo de modelo que reciba los parámetros adecuados para cada uno y devuelva el modelo entrenado y cualquier otra información necesaria y relevante
Por ejemplo para SVM:




```
def train_SVM(X,Y, C,lr ):
  …
  …
  return svm_model
```





### Arbol de Decisión con sklearn (Función Simple)

In [102]:
def train_decTree(X_train,y_train,max_depth_par,filename):
  global dt
  dt = DecisionTreeClassifier(criterion = 'entropy', max_depth = max_depth_par, random_state = 0)
  dt.fit(X_train, y_train)
  print('[5]Decision Tree Classifier Training Accuracy:', dt.score(X_train, y_train))
  # save the model to disk
  filename = ('/content/drive/MyDrive/Statistical Learning O/proyecto_final/models/%s.csv' % (filename))
  joblib.dump(dt, filename)
  return dt



In [103]:
max_depth_par = 3
filename = 'test1'

train_decTree(X_train,y_train,max_depth_par,filename)
Y_val_dt = dt.predict(X_val)
dt_accuracy = accuracy_score(y_val,Y_val_dt)
print('decision tree validation score:',dt_accuracy )
#joblib.dump(model, 'svm_lr=0.01_reg=0.1_var1_var2_var3.pkl')


[5]Decision Tree Classifier Training Accuracy: 0.7303370786516854
decision tree validation score: 0.7640449438202247


0.6256983240223464


### SVM con sklearn (Función Simple)

In [92]:
def train_SVM(X_train,y_train,random_state_var):
  global svc_lin
  svc_lin = SVC(kernel = 'linear', random_state = random_state_var )
  svc_lin.fit(X_train, y_train)
  print('[2]Support Vector Machine (Linear Classifier) Training Accuracy:', svc_lin.score(X_train, y_train))
  return svc_lin
  



In [93]:
random_state_var = 0
train_SVM(X_train,y_train,random_state_var)
Y_val_svm = svc_lin.predict(X_val)
svm_accuracy = accuracy_score(y_val,Y_val_svm)
print('SVM validation score:',svm_accuracy )


[2]Support Vector Machine (Linear Classifier) Training Accuracy: 0.7059925093632958
SVM validation score: 0.7640449438202247


### Naive Bayes con numpy y/o Pandas

In [32]:
def train_naiveBayese(X,y, epochs,lr ):
  …
  …
  return naiveBayes_model

SyntaxError: ignored

In [37]:
class NaiveBayes:
    def fit(self, X, y):
        n_samples, n_features = X.shape
        self._classes = np.unique(y)
        n_classes = len(self._classes)

        # calculate mean, var, and prior for each class
        self._mean = np.zeros((n_classes, n_features), dtype=np.float64)
        self._var = np.zeros((n_classes, n_features), dtype=np.float64)
        self._priors = np.zeros(n_classes, dtype=np.float64)

        for i, c in enumerate(self._classes):
            X_c = X[y == c]
            self._mean[i, :] = X_c.mean(axis=0)
            self._var[i, :] = X_c.var(axis=0)
            self._priors[i] = X_c.shape[0] / float(n_samples)

    def predict(self, X):
        y_pred = [self._predict(x) for x in X]
        return np.array(y_pred)

    def _predict(self, x):
        posteriors = [] 

        # calculate posterior probability for each class
        for i, c in enumerate(self._classes):
            prior = np.log(self._priors[i])
            posterior = np.sum(np.log(self._pdf(i, x)))
            posterior = prior + posterior
            posteriors.append(posterior)

        # return class with highest posterior probability
        return self._classes[np.argmax(posteriors)]

    def _pdf(self, class_idx, x):
        mean = self._mean[class_idx]
        var = self._var[class_idx]
        numerator = np.exp(-((x - mean) ** 2) / (2 * var))
        denominator = np.sqrt(2 * np.pi * var)
        return numerator / denominator
    
    def accuracy(y_true, y_pred):
        accuracy = np.sum(y_true == y_pred) / len(y_true)
        return accuracy
    


In [38]:
# Testing


nb = NaiveBayes()
nb.fit(X_train, y_train)
predictions = nb.predict(X_test)

print("Naive Bayes classification accuracy", accuracy(y_test, predictions))

NameError: ignored

In [ ]:
 epochs =
 lr = 

train_naiveBayese(X_train,y_train,random_state_var,epochs,lr)
Y_val_naiveBayese = naiveBayes_model.predict(X_val)
naiveBayes_model_accuracy = accuracy_score(y_val,Y_val_svm)
print('SVM validation score:',svm_accuracy )

### Reg. logística binaria(sigmoid)

In [ ]:
def train_logReg(X,y, C,lr ):
  initializer = tf.contrib.layers.xavier_initializer()
  h0 = tf.layers.dense(X, hidden_layers, activation=tf.nn.relu, kernel_initializer=initializer)
  # h0 = tf.nn.dropout(h0, 0.95)
  h1 = tf.layers.dense(h0, label_count, activation=None)

  cross_entropy = tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=h1)
  cost = tf.reduce_mean(cross_entropy)
  optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

  # prediction = tf.argmax(h0, 1)
  # correct_prediction = tf.equal(prediction, tf.argmax(Y_one_hot, 1))
  # accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

  predicted = tf.nn.sigmoid(h1)
  correct_pred = tf.equal(tf.round(predicted), Y)
  accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))


  return logReg_model

In [ ]:
# inputs
training_epochs = 3000
learning_rate = 0.01
hidden_layers = feature_count - 1
cost_history = np.empty(shape=[1],dtype=float)

X = tf.placeholder(tf.float32,[None,feature_count])
Y = tf.placeholder(tf.float32,[None,label_count])
is_training=tf.Variable(True,dtype=tf.bool)

### Ensemble learning model

In [35]:
def models(X_train,y_train,epochs,max_depth_par):
    
  #Using SVM Classifier
  def train_SVM(X_train,y_train):
    svc_lin = SVC(kernel = 'linear', random_state = 0 )
    svc_lin.fit(X_train, y_train)
    print('[2]Support Vector Machine (Linear Classifier) Training Accuracy:', svc_lin.score(X_train, y_train))
    return svc_lin

  #Using DecisionTreeClassifier
  def train_decTree(X_train,y_train,max_depth_par):
    dt = DecisionTreeClassifier(criterion = 'entropy', max_depth = 3, random_state = 0)
    dt.fit(X_train, y_train)
    print('[5]Decision Tree Classifier Training Accuracy:', dt.score(X_train, y_train))
    return dt



  #Returning Model accuracy by step
  print('SVM Training Accuracy:', svc_lin.score(X_train, y_train))
  print('Decision Tree Classifier Training Accuracy:', dt.score(X_train, y_train))
  


In [ ]:
class models2:
      
  #Using SVM Classifier
  def train_SVM(self,X_train,y_train):
    self.svc_lin = SVC(kernel = 'linear', random_state = 0 )
    svc_lin.fit(X_train, y_train)
    print('[2]Support Vector Machine (Linear Classifier) Training Accuracy:', svc_lin.score(X_train, y_train))
    return self.svc_lin

  #Using DecisionTreeClassifier
  def train_decTree(self,X_train,y_train,max_depth_par):
    self.dt = DecisionTreeClassifier(criterion = 'entropy', max_depth = 3, random_state = 0)
    dt.fit(X_train, y_train)
    print('[5]Decision Tree Classifier Training Accuracy:', dt.score(X_train, y_train))
    return self.dt



  #Returning Model accuracy by step
  print('SVM Training Accuracy:', svc_lin.score(X_train, y_train))
  print('Decision Tree Classifier Training Accuracy:', dt.score(X_train, y_train))

In [36]:
models = models(X_train,y_train,epochs = 0,max_depth_par=4)

SVM Training Accuracy: 0.7059925093632958
Decision Tree Classifier Training Accuracy: 0.7303370786516854


Cada vez que se entrena un modelo ( si se usan funciones como la anterior, cada vez que se ejecuta una de estas funciones) corresponde a un experimento, y para cada experimento(o llamada a una de las funciones):


Debemos crear la cadena de configuración que describa cómo se realizó cada experimento,  variables usadas, valores de hyper-parámetros y tipo de modelo usado (similar al config string que hemos usado en tensorboard)    por ejemplo:
Se entrena un regresión .logística con lr=0.01 ,factor de regularización = 0.1 usando las variables variables var1,var2,var3, la cadena podría ser: regLog_lr=0.01_reg=0.1_var1_var2_var3
Esta cadena nos servirá después para identificar cada experimento, y anotar en una bitácora( excel o csv)  las métricas de evaluación de cada uno.
Cada uno de los 4 tipos de modelos pueden requerir más de 1 experimento, con distintos hyper-parámetros o variables usadas, diagnosticamos overfitting u underfitting y realizamos acciones para atacarlos.
Por cada experimento debemos guardar en un excel o csv la cadena de configuración y las métricas de evaluación : accuracy,error, precisión,recall,f1-score , estas serán evaluados en el dataset de entrenamiento y el de validación. Es posible usar sklearn(o cualquier otro método que facilite el cálculo) para las métricas de evaluación y Pandas para guardar el csv o excel.
Este excel o csv no debe ser sobreescrito  en cada experimento si no que deben agregarse nuevas filas, esta será la bitácora de experimentos. 
Por cada experimento debemos guardar el modelo correspondiente identificandolo con su cadena de configuración, por ejemplo :
con sklearn para guardar un modelo "model" del experimento svm_lr=0.01_reg=0.1_var1_var2_var3 se puede usar:
		from sklearn.externals import joblib
	
		joblib.dump(model, 'svm_lr=0.01_reg=0.1_var1_var2_var3.pkl')
Con tensorflow podemos ejecutar en la session los parámetros entrenables(al terminar el entrenamiento) y asignarlos a un tensor de Numpy llamado W,
luego podemos guardar el tensor de parámetros del modelo  con  dump()
W = session.run(parametros,feed_dict).
(Tensorflow tiene sus propios métodos de export y deploy, convenientes para modelos mas complejos (como modelos profundos) , se debe investigar y anotar en markdown conclusiones sobre este tema, en caso de aplicarlas se tomará como puntos extra).


W.dump("regLog_lr=0.01_reg=0.1_var1_var2_var3.npy")
 Con numpy (para el clasificador de bayes) podemos también usar dump (o si se usa pandas usar to_csv por ejemplo)


Luego de haber ejecutado ,evaluado, y guardado varios experimentos,elegimos para cada tipo de modelo el mejor basado en las métricas de evaluación sobre los datos de validación. (recordar la exactitud deseada)


## Investigar la técnica: k-fold cross validation 

agregando en markdown una descripción de esta y como se pudo haber aplicado en este proyecto (No debemos aplicarla al proyecto , solo describir la técnica y cómo se podría aplicar) . 

Por ejemplo: https://towardsdatascience.com/why-and-how-to-cross-validate-a-model-d6424b45261f . 

Si esta es aplicada se tomará como puntuación extra


In [ ]:
kfold = model_selection.KFold(n_splits=10, random_state=seed)
# create the sub models
estimators = []
model1 = LogisticRegression()
estimators.append(('logistic', model1))
model2 = DecisionTreeClassifier()
estimators.append(('cart', model2))
model3 = SVC()
estimators.append(('svm', model3))
# create the ensemble model
ensemble = VotingClassifier(estimators)
results = model_selection.cross_val_score(ensemble, X, Y, cv=kfold)
print(results.mean())

In [85]:
def modelVoting (X_train,y_train,max_depth_par):
  
  #Appying Kfold
  seed = 7
  kfold = model_selection.KFold(n_splits=10)                              
  #Creating dictionary of Models 
  estimators = [] 

  #Using SVM Classifier
  def train_SVM(X_train,y_train):
    svc_lin = SVC(kernel = 'linear')
    svc_lin.fit(X_train, y_train)
    return svc_lin

  #Using DecisionTreeClassifier
  def train_decTree(X_train,y_train,max_depth_par):
    dt = DecisionTreeClassifier(criterion = 'entropy', max_depth = 3)
    dt.fit(X_train, y_train)
    return dt

  #Using GaussianNB method of naïve_bayes
  #def train_naive_bayes(X_train,y_train):
   # nb = GaussianNB()
   # nb.fit(X_train, Y_train)
   # return nb

  #Using Logistic Regression Algorithm to the Training Set
  #def logReg(X_train,y_train):
    #logReg = LogisticRegression()
    #logReg.fit(X_train, Y_train)
    #return logReg

  # create the ensemble model
  estimators.append(('svm', svc_lin))
  estimators.append(('dt', dt))
  ensemble = VotingClassifier(estimators)
  #estimators.append(('naive_bayes', nb))
  #estimators.append(('log', logReg))
  results = model_selection.cross_val_score(ensemble, X_train, y_train, cv=kfold)
  print(results.mean())

In [87]:
modelVoting (X_train,y_train,3)

0.6910202655485673


## Prueba/Evaluación final:

Dada el conjunto de observaciones X de el conjunto de pruebas y los 4 modelos elegidos:
	
Predecir sobre estas usando el mejor modelo de cada tipo elegido en el punto anterior.
 Combinar los resultados de las predicciones en una predicción final(moda de resultados individuales)
Generar una tabla de predicciones como el ejemplo x y crear un dataframe de Pandas con los resultados.
Calcular métricas de evaluación comparando los Y reales del conjunto de pruebas, contra él Y que se obtuvo de combinar las predicciones individuales del modelo.
Similar al paso anterior, generar una tabla de métricas de evaluación y crear un dataframe de pandas para mostrar el resultado final.


Si en la evaluación final, no se obtiene la exactitud mínima deseada, volver a la fase de experimentación .


## Conclusiones
Agregar sección de conclusiones y recomendaciones (incluyendo opiniones,experiencias ,dificultades y lecciones aprendidas)
